In [50]:
!pip install bert-tensorflow==1.0.1
%tensorflow_version 1.x
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import bert
import math
from bert import run_classifier
from google.colab import drive
drive.mount('/content/drive')
OUTPUT_DIR = '/content/drive/My Drive/6000/fine-tuning1'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This is a path to a chinese version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],tokenization_info["do_lower_case"]])    
  return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()
BATCH_SIZE = 32
MAX_SEQ_LENGTH = 128
label_list = [0,1,2,3,4]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=False)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)
  
  output_layer = bert_outputs["pooled_output"]
  hidden_size = output_layer.shape[-1].value
  output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],initializer=tf.truncated_normal_initializer(stddev=0.02))
  output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.5)
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))

    return (predicted_labels, log_probs)

def model_fn_builder(num_labels):
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
    predictions = {
        'probabilities': log_probs,
        'labels': predicted_labels
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
  return model_fn

In [ ]:
run_config = tf.estimator.RunConfig(model_dir=OUTPUT_DIR)
model_fn = model_fn_builder(num_labels=len(label_list))
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/6000/fine-tuning1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f31e0144208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/6000/fine-tuning1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f31e0144208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [53]:
labels = ["Extremely Negative", "Negative", "Neutral", "Positive", "Extremely Positive"]
def getPrediction(in_sentences):
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [54]:
file = pd.read_csv('/content/drive/My Drive/6000/test.csv')
label = file.values
confusion = np.zeros((5,5))
predictions = getPrediction(label[:,0])
pred = []
for i in range(len(label)):
  label[i,0] = predictions[i][2]
  if label[i,0] == 'Extremely Positive':
    confusion[label[i,1],4]+=1
    pred.append(4)
  elif label[i,0] == 'Positive':
    confusion[label[i,1],3]+=1
    pred.append(3)
  elif label[i,0] == 'Neutral':
    confusion[label[i,1],2]+=1
    pred.append(2)
  elif label[i,0] == 'Negative':
    confusion[label[i,1],1]+=1
    pred.append(2)
  elif label[i,0] == 'Extremely Negative':
    confusion[label[i,1],0]+=1
    pred.append(0)
  else:
    print('wrong')
auc = (confusion[0,0]+confusion[1,1]+confusion[2,2]+confusion[3,3]+confusion[4,4])/len(label)
print(confusion)
print(auc)
print(len(label))

INFO:tensorflow:Writing example 0 of 3798


INFO:tensorflow:Writing example 0 of 3798


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


[[526.  58.   3.   5.   0.]
 [ 53. 906.  43.  33.   6.]
 [  1.  34. 556.  27.   1.]
 [  3.  49.  50. 801.  44.]
 [  0.   2.   1.  81. 515.]]
0.8699315429173249
3798


In [ ]:
df = file
df['prediction']=pred
print(df)
df.to_csv('/content/drive/My Drive/6000/pred.csv',index=False)

                                                   text  emotion  prediction
0     TRENDING: New Yorkers encounter empty supermar...        0           0
1     When I couldn't find hand sanitizer at Fred Me...        3           3
2     Find out how you can protect yourself and love...        4           4
3     #Panic buying hits #NewYork City as anxious sh...        1           2
4     #toiletpaper #dunnypaper #coronavirus #coronav...        2           2
...                                                 ...      ...         ...
3793  Meanwhile In A Supermarket in Israel -- People...        3           3
3794  Did you panic buy a lot of non-perishable item...        1           2
3795  Asst Prof of Economics @cconces was on @NBCPhi...        2           2
3796  Gov need to do somethings instead of biar je r...        0           0
3797  I and @ForestandPaper members are committed to...        4           4

[3798 rows x 3 columns]


In [55]:
pred_sentences = [
  "Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need. The prices are pretty reasonable in most of them panicbuying.",
  "Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need.",
  "The prices are pretty reasonable in most of them panicbuying.",
  ]
predict = getPrediction(pred_sentences)
predict
# ground truth: Neutral

INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:Writing example 0 of 3


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


[('Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need. The prices are pretty reasonable in most of them panicbuying.',
  array([-6.6535196 , -1.0338495 , -0.58121115, -2.4979064 , -6.4392824 ],
        dtype=float32),
  'Neutral'),
 ('Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need.',
  array([-5.251242  , -0.01375952, -5.9529114 , -5.6100993 , -6.1351533 ],
        dtype=float32),
  'Negative'),
 ('The prices are pretty reasonable in most of them panicbuying.',
  array([-8.3756552e+00, -7.9211841e+00, -7.5499368e+00, -4.4358191e-03,
         -5.7118793e+00], dtype=float32),
  'Positive')]

In [62]:
for i in range(len(predict)):
  print('sentence: '+ str(predict[i][0]))
  print('predicted label: '+str(predict[i][2]))
  for j in range(5):
    print("{} {:.4f}".format(labels[j],math.exp(predict[i][1][j])))
  print()

sentence: Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need. The prices are pretty reasonable in most of them panicbuying.
predicted label: Neutral
Extremely Negative 0.0013
Negative 0.3556
Neutral 0.5592
Positive 0.0823
Extremely Positive 0.0016

sentence: Instead of panic buying in the crowded supermarkets, pop down to the local shop and see if they have what you need.
predicted label: Negative
Extremely Negative 0.0052
Negative 0.9863
Neutral 0.0026
Positive 0.0037
Extremely Positive 0.0022

sentence: The prices are pretty reasonable in most of them panicbuying.
predicted label: Positive
Extremely Negative 0.0002
Negative 0.0004
Neutral 0.0005
Positive 0.9956
Extremely Positive 0.0033



In [56]:
pred_sentences1 = [
  "My country has gone into a lockdown. Stay safe everyone wash your hands daily, use hand sanitizer and make sure elders have spare food in shops! Panic spree shopping left elders with very little food ?",
  "My country has gone into a lockdown.",
  "Stay safe everyone wash your hands daily, use hand sanitizer and make sure elders have spare food in shops!",
  "Panic spree shopping left elders with very little food ?"
  ]
predict1 = getPrediction(pred_sentences1)
# ground truth: Extremely Positive

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/6000/fine-tuning1/model.ckpt-3857


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [61]:
for i in range(len(predict1)):
  print('sentence: '+ str(predict1[i][0]))
  print('predicted label: '+str(predict1[i][2]))
  for j in range(5):
    print("{} {:.4f}".format(labels[j],math.exp(predict1[i][1][j])))
  print()

sentence: My country has gone into a lockdown. Stay safe everyone wash your hands daily, use hand sanitizer and make sure elders have spare food in shops! Panic spree shopping left elders with very little food ?
predicted label: Positive
Extremely Negative 0.0012
Negative 0.0009
Neutral 0.0008
Positive 0.4985
Extremely Positive 0.4985

sentence: My country has gone into a lockdown.
predicted label: Neutral
Extremely Negative 0.0003
Negative 0.0009
Neutral 0.9969
Positive 0.0017
Extremely Positive 0.0002

sentence: Stay safe everyone wash your hands daily, use hand sanitizer and make sure elders have spare food in shops!
predicted label: Extremely Positive
Extremely Negative 0.0015
Negative 0.0010
Neutral 0.0009
Positive 0.0010
Extremely Positive 0.9957

sentence: Panic spree shopping left elders with very little food ?
predicted label: Negative
Extremely Negative 0.0311
Negative 0.9646
Neutral 0.0015
Positive 0.0020
Extremely Positive 0.0008

